In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337)

In [ ]:
flagD = 0
th = 24
ts = 30
NSteps = th*3600/ts
NSteps = int(NSteps)

# SBML model we want to import
sbml_file = 'SPARCED_Brep.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

kGsRead = pd.read_csv('OmicsData.txt',header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
# # Make serum-fed population of random cells
numStocCells = 20
STIMligs = [3.3,100.0,100.0,100.0,100.0,100.0,1721.0] # GM cond
namexlsfile = 'Fig3H_20cells_'  

startTime = datetime.now()
print(startTime)

Vn = 1.75E-12 
Vc = 5.25E-12 
spdata0 = pd.read_csv('Species_Brep.txt',header=0,index_col=0,sep="\t")
spdata = np.double(spdata0.values[:,1])
spdata[155:162] = STIMligs
genedata0, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)
tout_all = np.arange(0,th*3600+1,30)
condsS = []
condsG = []
for nn in range(numStocCells): 
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
    xoutS_all[0,:] = spdata
    xoutG_all = np.zeros(shape=(NSteps+1,len(genedata0)))
    xoutG_all[0,:] = genedata0
    for qq in range(NSteps):
        genedata,AllGenesVec = gm(flagD,ts,xoutG_all[qq,:],xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model, solver)  # Run simulation
        if rdata['x'][-1,103] < rdata['x'][-1,105]:
            print('Apoptosis happened')
            xoutS_all[qq+1,:] = rdata['x'][-1,:]
            xoutG_all[qq+1,:] = genedata
            break
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
        xoutG_all[qq+1,:] = genedata
    xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
    condsS.append(xoutS_all)
    xoutG_all = xoutG_all[~np.all(xoutG_all == 0, axis=1)]
    condsG.append(xoutG_all)    
    condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model.getStateIds()]) 
    condsSDF.to_csv(namexlsfile+'S_'+str(nn)+'.txt',sep="\t")    
    condsSDF = None
    condsGDF = pd.DataFrame(data=xoutG_all) 
    condsGDF.to_csv(namexlsfile+'G_'+str(nn)+'.txt',sep="\t")    
    condsGDF = None    
    print(datetime.now() - startTime)
print(datetime.now())

In [ ]:
# Do TRAIL dose response - starts by (importing) random stochastic cells
startTime = datetime.now()
print(startTime)

flagD = 0
th = 5
numStocCells = 20
traildoses = [0.0385, 0.1925, 0.385, 1.9250, 3.85, 19.25, 38.5]
STIMligs = [3.3,100.0,100.0,100.0,100.0,100.0,1721.0] # GM cond

ts = 30
NSteps = th*3600/ts
NSteps = int(NSteps)
tout_all = np.arange(0,th*3600+1,30)
Vn = 1.75E-12
Vc = 5.25E-12

condsSTR = []
condsGTR = []
namexlsfile = 'Fig3H_20cells_TRAIL_'   
for tt in range(len(traildoses)):
    for cc in range(len(condsS)):  # len(condsS)
        spdata2 = condsS[cc][-1,:]
        spdata2[83] = traildoses[tt]
        spdata2[155:162] = STIMligs
        xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
        xoutS_all[0,:] = spdata2
        xoutG_all = np.zeros(shape=(NSteps+1,len(genedata)))
        xoutG_all[0,:] = condsG[cc][-1,:]
        for qq in range(NSteps):
            genedata,AllGenesVec = gm(flagD,ts,xoutG_all[qq,:],xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                       kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
            for ii,kk in enumerate(kmRNAs):
                model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
            model.setInitialStates(xoutS_all[qq,:])
            rdata = amici.runAmiciSimulation(model, solver)  # Run simulation
            xoutS_all[qq+1,:] = rdata['x'][-1,:]
            xoutG_all[qq+1,:] = genedata
            if rdata['x'][-1,103] < rdata['x'][-1,105]:
                print('Apoptosis happened')
                break            
        xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
        condsSTR.append(xoutS_all)
        xoutG_all = xoutG_all[~np.all(xoutG_all == 0, axis=1)]
        condsGTR.append(xoutG_all)    
        condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model.getStateIds()]) 
        condsSDF.to_csv(namexlsfile+'S_'+str(tt)+'_'+str(cc)+'.txt',sep="\t")    
        condsSDF = None
        condsGDF = pd.DataFrame(data=xoutG_all) 
        condsGDF.to_csv(namexlsfile+'G_'+str(tt)+'_'+str(cc)+'.txt',sep="\t")    
        condsGDF = None
        print(datetime.now() - startTime)
print(datetime.now())

In [ ]:
numStocCells = 20
traildoses = [0.0385, 0.1925, 0.385, 1.9250, 3.85, 19.25, 38.5]
namexls = 'Fig3H_20cells_TRAIL_'
condsS = []
dead_cells = []
doses = []
percent_dead_cells = []
count1 = 0
for tt in range(len(traildoses)):
    for cc in range(numStocCells):
        condsDF = pd.read_csv(namexls+'S_'+str(tt)+'_'+str(cc)+'.txt',header=0,index_col=0,sep="\t")
        condsDF = np.double(condsDF.values[:])
        condsS.append(condsDF)
        condsDF = None
        yy = np.array(condsS[count1][:,105])
        count1 += 1
        if any(yy>100):
            dead_cells.append(1.0)
        else:
            dead_cells.append(0.0)
    doses.append(condsS[-1][0,83])
    aa = ((tt+1)*20)-20
    bb = (tt+1)*20
    percent_dead_cells.append((sum(dead_cells[aa:bb])/20.0)*100.0)

In [ ]:
numStocCells = 20
traildoses = [0.0385, 0.1925, 0.385, 1.9250, 3.85, 19.25, 38.5]
dead_cells = []
percent_dead_cells = []
count1 = 0
for tt in range(len(traildoses)):
    for pp in range(10):
        yy = np.array(condsS[count1][:,105])
        count1 += 1
        if any(yy>90):
            dead_cells.append(1.0)
        else:
            dead_cells.append(0.0)     
    aa = ((tt+1)*20)-20
    bb = (tt+1)*20
    percent_dead_cells.append((sum(dead_cells[aa:bb])/10.0)*100.0)

In [ ]:
dosesngperml = np.array(doses)*2.597402597402597e+01
trailexpdoses = [2.34375,4.6875,9.375,18.75,37.5,75.0,150.0,300.0] # From Bouhaddou2018 model paper
trailexppdeath = [89.79,93.53,88.40,79.53,58.08,36.34,20.18,21.11] # From Bouhaddou2018 model paper

plt.figure(figsize=(7, 4))
plt.plot(np.log10(np.array(trailexpdoses)),np.array(trailexppdeath),marker='*',MarkerSize=10) # Experimental
plt.plot(np.log10(np.array(dosesngperml)),100.0-np.array(percent_dead_cells),marker='o',MarkerSize=10) # Simulations
plt.xlabel('TRAIL (ng/mL)', multialignment='center')
plt.ylabel('% Surviving cells')
plt.grid(True)
# plt.xlim(0, 72)
plt.ylim(0, 120)
plt.xticks(np.arange(0,4,step=1),('10e0', '10e1', '10e2', '10e3'))
plt.legend(['Experimental','Simulations'])
plt.savefig('cPARP_Fig3H.png')